## ref컷 생성하기
------------------------------------------------------
## STEP 1 (batch API에 집어넣을 jsonl 생성)

In [ ]:
import os
import json
import random
import hashlib
from io import BytesIO
from PIL import Image, ImageOps, ImageStat
from google.cloud import storage

# ==============================================================================
# [1. 설정] 버킷 내 경로로 수정해주세요!
# ==============================================================================

BUCKET_NAME = "rawer_asac"

# [버킷 내 폴더 경로] (gs://rawer_asac/ 뒤에 오는 경로들)
# 예: gs://rawer_asac/bottles/ 라면 "bottles/" 라고 적습니다.
GCS_BOTTLES_PREFIX = "bottles/"
GCS_LOGOS_PREFIX = "logos/"
GCS_DATASET_PREFIX = "final_font_dataset/" # 이 안에 front_bw, back_bw 등이 있다고 가정

# [결과 파일명]
OUTPUT_JSONL = "batch_input.jsonl"
DARK_THRESHOLD = 70

# ==============================================================================
# [2. 유틸리티 함수]
# ==============================================================================

def list_gcs_files(bucket, prefix):
    """GCS 버킷의 특정 폴더(prefix) 내 모든 이미지 파일 목록(Blob 객체)을 가져옴"""
    # prefix가 폴더임을 명시하기 위해 끝에 / 가 없으면 붙여줌
    if prefix and not prefix.endswith('/'): prefix += '/'

    blobs = bucket.list_blobs(prefix=prefix)
    image_blobs = []
    for blob in blobs:
        if blob.name.lower().endswith(('.png', '.jpg', '.jpeg', '.webp')):
            image_blobs.append(blob)
    return image_blobs

def download_image_to_memory(blob):
    """GCS Blob 데이터를 메모리(BytesIO)로 다운로드하여 PIL 이미지로 변환"""
    img_bytes = blob.download_as_bytes()
    return Image.open(BytesIO(img_bytes))

def upload_processed_image(bucket, pil_img, original_blob_name):
    """
    메모리 상의 PIL 이미지(반전된 폰트 등)를 GCS의 'processed/' 폴더에 업로드하고 URI 반환
    """
    # 파일명 해시 생성 (중복 방지)
    img_byte_arr = BytesIO()
    pil_img.save(img_byte_arr, format='PNG')
    img_bytes = img_byte_arr.getvalue()
    file_hash = hashlib.md5(img_bytes).hexdigest()[:10]

    # 원본 파일명 추출
    base_name = os.path.basename(original_blob_name)
    new_blob_name = f"processed_fonts/{os.path.splitext(base_name)[0]}_{file_hash}.png"

    blob = bucket.blob(new_blob_name)
    if not blob.exists():
        print(f"   ⬆️ (가공됨) 업로드: {new_blob_name}")
        blob.upload_from_string(img_bytes, content_type="image/png")

    return f"gs://{BUCKET_NAME}/{new_blob_name}"

def is_dark_bottle(pil_img):
    try:
        stat = ImageStat.Stat(pil_img.convert("L"))
        return stat.mean[0] < DARK_THRESHOLD
    except: return False

def smart_invert_font(pil_img):
    try:
        if pil_img.mode == 'RGBA':
            r, g, b, a = pil_img.split()
            rgb = Image.merge('RGB', (r, g, b))
            inv = ImageOps.invert(rgb)
            r2, g2, b2 = inv.split()
            return Image.merge('RGBA', (r2, g2, b2, a))
        else:
            return ImageOps.invert(pil_img.convert("RGB"))
    except: return pil_img

# ==============================================================================
# [3. 메인 로직]
# ==============================================================================

def main():
    print(f"🚀 [GCS 모드] 버킷({BUCKET_NAME})에서 데이터를 읽어 작업을 준비합니다...")

    client = storage.Client()
    bucket = client.bucket(BUCKET_NAME)

    # 1. 공병/로고 목록 가져오기 (GCS List)
    print("   🔎 공병 및 로고 목록 조회 중...")
    bottle_blobs = list_gcs_files(bucket, GCS_BOTTLES_PREFIX)
    logo_blobs = list_gcs_files(bucket, GCS_LOGOS_PREFIX)

    if not bottle_blobs:
        print(f"❌ 오류: '{GCS_BOTTLES_PREFIX}' 경로에 이미지가 없습니다."); return

    # 2. 데이터셋 폴더 순회
    # GCS는 폴더 개념이 없으므로 prefix로 검색해야 합니다.
    # 사용자가 말한 4개 폴더명을 prefix로 조합합니다.
    target_subfolders = ["front_bw", "front_color_aug", "back_bw", "back_color_aug"]

    jsonl_entries = []

    for subfolder in target_subfolders:
        # 예: final_font_dataset/front_bw/
        full_prefix = f"{GCS_DATASET_PREFIX.rstrip('/')}/{subfolder}/"

        font_blobs = list_gcs_files(bucket, full_prefix)
        if not font_blobs:
            print(f"⚠️ 건너뜀: '{full_prefix}' (파일 없음)")
            continue

        print(f"📂 폴더 처리 중: {subfolder} ({len(font_blobs)}장)")

        view_type = "back" if "back" in subfolder.lower() else "front"

        for font_blob in font_blobs:
            # --- [A] 랜덤 매칭 ---
            selected_bottle_blob = random.choice(bottle_blobs)
            selected_logo_blob = random.choice(logo_blobs) if logo_blobs else None

            # 기본 URI (가공 안 할 경우 이거 씀)
            uri_bottle = f"gs://{BUCKET_NAME}/{selected_bottle_blob.name}"
            uri_font = f"gs://{BUCKET_NAME}/{font_blob.name}"
            uri_logo = f"gs://{BUCKET_NAME}/{selected_logo_blob.name}" if selected_logo_blob else None

            # --- [B] 이미지 분석 (메모리 다운로드) ---
            # 1) 병 이미지가 어두운지 확인해야 함 -> 다운로드 필요
            # (매번 다운로드하면 느리니 캐싱하면 좋지만, 간단히 구현)
            img_bottle = download_image_to_memory(selected_bottle_blob)
            is_dark = is_dark_bottle(img_bottle)

            # 2) 조건 맞으면 폰트 다운로드 & 반전 처리
            is_inverted = False
            if is_dark and "bw" in subfolder.lower():
                img_font = download_image_to_memory(font_blob)
                img_font_inverted = smart_invert_font(img_font)

                # 반전된 이미지를 GCS에 'processed_fonts/'로 업로드하고, 그 주소로 교체
                uri_font = upload_processed_image(bucket, img_font_inverted, font_blob.name)
                is_inverted = True

            # --- [C] 프롬프트 작성 ---
            text_inst = "Text Color: Use WHITE/Light color for high contrast." if is_inverted else "Text Color: Follow Ref C strictly."

            prompt = f"""
You are an expert product photographer using the 'Gemini 3 Pro' engine.
VIEW: {view_type.upper()}
TASK: Composite the provided typography (Ref C) onto the product container (Ref A).
{f'LOGO: Place Ref B logo on upper center.' if view_type == 'front' else 'LOGO: DO NOT include any logo.'}
TEXT: Apply Ref C text naturally on the surface. {text_inst}
OUTPUT: High-quality photorealistic image.
"""

            # --- [D] JSONL 엔트리 생성 ---
            parts = [{"text": prompt}]
            parts.append({"file_data": {"mime_type": "image/png", "file_uri": uri_bottle}})
            if uri_logo and view_type == "front":
                parts.append({"file_data": {"mime_type": "image/png", "file_uri": uri_logo}})
            parts.append({"file_data": {"mime_type": "image/png", "file_uri": uri_font}})

            entry = {
                "custom_id": f"{subfolder}_{os.path.basename(font_blob.name)}",
                "request": {
                    "contents": [{"role": "user", "parts": parts}],
                    "generation_config": {
                        "response_modalities": ["IMAGE"],
                        "temperature": 0.3
                    }
                }
            }
            jsonl_entries.append(entry)

    # JSONL 저장
    with open(OUTPUT_JSONL, "w", encoding="utf-8") as f:
        for e in jsonl_entries:
            f.write(json.dumps(e) + "\n")

    print(f"\n🎉 [완료] 총 {len(jsonl_entries)}개의 작업 준비 끝!")
    print(f"👉 생성된 '{OUTPUT_JSONL}' 파일을 GCS에 올리고 Step 2를 실행하세요.")

if __name__ == "__main__":
    main()

## STEP 2 (생성 요청)

In [ ]:
from google import genai
import os

# ==============================================================================
# [설정] Step 1과 동일하게 맞춰주세요
# ==============================================================================

BUCKET_NAME = "rawer_asac"       # 사용자님 버킷 이름
LOCATION = "us-central1"         # Batch API는 us-central1 사용 권장
MODEL_ID = "gemini-3-pro-image-preview" # 사용할 모델

# [중요] Step 1에서 만든 파일을 버킷에 올렸을 때의 경로
INPUT_URI = f"gs://{BUCKET_NAME}/batch_input.jsonl"

# 결과가 저장될 버킷 내 폴더 경로
OUTPUT_URI = f"gs://{BUCKET_NAME}/results/"

# ==============================================================================
# [메인 로직]
# ==============================================================================

def submit_job():
    print(f"🚀 [Step 2] Gemini Batch 작업을 제출합니다...")
    print(f"   - 모델: {MODEL_ID}")
    print(f"   - 입력 파일: {INPUT_URI}")
    print(f"   - 결과 저장소: {OUTPUT_URI}")

    # 클라이언트 연결
    try:
        client = genai.Client(location=LOCATION)
    except Exception as e:
        print(f"❌ 클라이언트 연결 실패: {e}")
        return

    # 작업 제출
    try:
        job = client.batches.create(
            model=MODEL_ID,
            src=INPUT_URI,
            dest=OUTPUT_URI
        )

        print("\n" + "="*50)
        print(f"✅ 작업 제출 성공! (Job Submitted)")
        print("="*50)
        print(f"🆔 Job ID: {job.name}")
        print(f"📂 상태 확인: Google Cloud Console > Batch 메뉴")
        print(f"⚠️ 주의: 작업이 완료되기까지 시간이 걸립니다 (몇 분~몇 시간)")
        print(f"   완료되면 {OUTPUT_URI} 경로에 결과 파일이 생성됩니다.")

    except Exception as e:
        print(f"\n❌ 작업 제출 실패 (에러 발생):")
        print(f"{e}")
        print("\n[체크리스트]")
        print("1. 'batch_input.jsonl' 파일을 버킷에 업로드 하셨나요?")
        print("2. 구글 클라우드 인증(gcloud auth ...)이 되어 있나요?")

if __name__ == "__main__":
    submit_job()